In [12]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
import chess
import numpy as np
import shelve
import os

In [23]:
def get_two_bishops_start():
    board = chess.Board()
    board.clear()
    board.set_piece_at(0, chess.Piece(chess.BISHOP, True))
    board.set_piece_at(1, chess.Piece(chess.BISHOP, True))
    board.set_piece_at(2, chess.Piece(chess.KING, True))
    board.set_piece_at(56, chess.Piece(chess.KING, False))
    board.turn = True
    return board

In [24]:
def get_rook_start():
    board = chess.Board()
    board.clear()
    board.set_piece_at(1, chess.Piece(chess.ROOK, True))
    board.set_piece_at(2, chess.Piece(chess.KING, True))
    board.set_piece_at(60, chess.Piece(chess.KING, False))
    board.turn = True
    return board

In [25]:
def get_state_from_board(board):
    state = ' '.join(board.fen().split(' ')[:2])
    return state

def get_next_state(board, move):
    board.push(move)
    fen = ' '.join(board.fen().split(' ')[:2])
    status = int(board.is_checkmate())
    board.pop()
    return fen, -status

In [26]:
import collections

In [36]:
def bfs(board, folder):
    state = get_state_from_board(board)
    seen = set([])
    if not os.path.exists(folder):
        os.makedirs(folder)
    states = shelve.open(folder+'/states', flag='n', writeback=True) 
    
    queue = collections.deque([state])
    while queue:
        state = queue.popleft()
        board = chess.Board()
        fen = state + ' - - 0 1'
        board.set_fen(fen)
        if board.is_game_over() == 0:
            states[state] = {}
            for move in board.legal_moves:
                action = str(move)
                states[state][action] = {}
                fen, status = get_next_state(board, move)
                states[state][action]['status'] = status
                states[state][action]['next_node'] = fen
                if fen not in seen:
                    seen.add(fen)
                    queue.append(fen)
        N = len(states)
        if N%10000 == 0:
            print('\rstates: {}'.format(N), end='')
            states.sync()
    print()
    states.close()
    return states

In [37]:
board = get_rook_start()
%time _ = bfs(board, './data/rook_states_tree')

states: 390000CPU times: user 4min 29s, sys: 6.71 s, total: 4min 36s
Wall time: 4min 36s


In [ ]:
board = get_two_bishops_start()
%time _ = bfs(board, './data/two_bishopw_states_tree')

states: 1710000